In [ ]:
# Load environment variables and create Anthropic client for API communication

from dotenv import load_dotenv
from anthropic import Anthropic

# Load API keys and other sensitive variables from .env file
load_dotenv()

# Initialize the Anthropic client; credentials are automatically read from environment
client = Anthropic()

# Specify the model to use for all chat API calls
model = "claude-haiku-4-5"


In [ ]:
# Helper functions for message management and chat operations

from anthropic.types import Message


def add_user_message(messages, message):
    """
    Add a user message to the conversation history.
    
    Args:
        messages: List of message dicts in the conversation
        message: User message (string or Anthropic Message object)
    """
    # Extract text content if message is an Anthropic Message object, otherwise use directly
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    """
    Add an assistant message to the conversation history.
    
    Args:
        messages: List of message dicts in the conversation
        message: Assistant message (string or Anthropic Message object, including tool results)
    """
    # Extract text content if message is an Anthropic Message object, otherwise use directly
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    """
    Send a chat request to the Anthropic API.
    
    Args:
        messages: List of message dicts with role and content
        system: Optional system prompt to guide model behavior
        temperature: Sampling temperature (higher = more random)
        stop_sequences: Strings where generation should stop
        tools: Optional list of tool schemas the model can use
    
    Returns:
        Anthropic Message object containing the response
    """
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    # Include tools parameter only if tools are provided
    if tools:
        params["tools"] = tools

    # Include system prompt only if provided
    if system:
        params["system"] = system

    # Make the API call and return the response message
    message = client.messages.create(**params)
    return message


def text_from_message(message):
    """
    Extract all text content from an Anthropic Message object.
    
    Args:
        message: Anthropic Message object with multiple content blocks
    
    Returns:
        String with all text blocks joined by newlines
    """
    # Filter for text blocks and join them with newlines
    return "\n".join([block.text for block in message.content if block.type == "text"])


In [ ]:
# Tool implementations and their corresponding JSON schemas for the Anthropic API

from datetime import datetime, timedelta


def add_duration_to_datetime(
    datetime_str, duration=0, unit="days", input_format="%Y-%m-%d"
):
    """
    Add a specified duration to a datetime string and return the result in a detailed format.
    
    Args:
        datetime_str: Input datetime string (e.g., '2025-04-03')
        duration: Amount of time to add (positive for future, negative for past)
        unit: Time unit - 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'
        input_format: Python strftime format string for parsing datetime_str
    
    Returns:
        Formatted string like 'Thursday, April 03, 2025 10:30:00 AM'
    """
    # Parse the input string into a datetime object
    date = datetime.strptime(datetime_str, input_format)

    # Add the duration in the specified unit
    if unit == "seconds":
        new_date = date + timedelta(seconds=duration)
    elif unit == "minutes":
        new_date = date + timedelta(minutes=duration)
    elif unit == "hours":
        new_date = date + timedelta(hours=duration)
    elif unit == "days":
        new_date = date + timedelta(days=duration)
    elif unit == "weeks":
        new_date = date + timedelta(weeks=duration)
    elif unit == "months":
        # Special handling for months (timedelta doesn't support month arithmetic)
        month = date.month + duration
        year = date.year + month // 12
        month = month % 12
        # Handle month wraparound (month 0 becomes month 12 of previous year)
        if month == 0:
            month = 12
            year -= 1
        # Adjust day if it exceeds the maximum for the target month
        day = min(
            date.day,
            [
                31,
                29 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 28,
                31,
                30,
                31,
                30,
                31,
                31,
                30,
                31,
                30,
                31,
            ][month - 1],
        )
        new_date = date.replace(year=year, month=month, day=day)
    elif unit == "years":
        new_date = date.replace(year=date.year + duration)
    else:
        raise ValueError(f"Unsupported time unit: {unit}")

    # Return the result in a detailed human-readable format
    return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")


def set_reminder(content, timestamp):
    """
    Create a timed reminder that notifies the user at the specified time.
    
    Args:
        content: Message text to display in the reminder
        timestamp: Date and time when reminder should trigger
    """
    # Print the reminder details (in production, this would persist to a database)
    print(f"----\nSetting the following reminder for {timestamp}:\n{content}\n----")


# Schema for the add_duration_to_datetime tool (for API documentation)
add_duration_to_datetime_schema = {
    "name": "add_duration_to_datetime",
    "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
    "input_schema": {
        "type": "object",
        "properties": {
            "datetime_str": {
                "type": "string",
                "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
            },
            "duration": {
                "type": "number",
                "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
            },
            "unit": {
                "type": "string",
                "description": "The unit of time for the duration. Must be one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Defaults to 'days'.",
            },
            "input_format": {
                "type": "string",
                "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
            },
        },
        "required": ["datetime_str"],
    },
}

# Schema for the set_reminder tool (for API documentation)
set_reminder_schema = {
    "name": "set_reminder",
    "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
    "input_schema": {
        "type": "object",
        "properties": {
            "content": {
                "type": "string",
                "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
            },
            "timestamp": {
                "type": "string",
                "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
            },
        },
        "required": ["content", "timestamp"],
    },
}

# Schema for a batch tool (allows invoking multiple tools simultaneously)
batch_tool_schema = {
    "name": "batch_tool",
    "description": "Invoke multiple other tool calls simultaneously",
    "input_schema": {
        "type": "object",
        "properties": {
            "invocations": {
                "type": "array",
                "description": "The tool calls to invoke",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "The name of the tool to invoke",
                        },
                        "arguments": {
                            "type": "string",
                            "description": "The arguments to the tool, encoded as a JSON string",
                        },
                    },
                    "required": ["name", "arguments"],
                },
            }
        },
        "required": ["invocations"],
    },
}

pass


In [ ]:
# get_current_datetime tool implementation and schema

from anthropic.types import ToolParam


def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
    """
    Return the current date and time formatted according to the specified format string.
    
    Args:
        date_format: Python strftime format string (default: '%Y-%m-%d %H:%M:%S')
    
    Returns:
        Current datetime as a formatted string
    
    Raises:
        ValueError: If date_format is empty or None
    """
    if not date_format:
        raise ValueError("date_format cannot be empty")
    
    # Get current system time and format it according to the provided format
    return datetime.now().strftime(date_format)


# Schema definition for the get_current_datetime tool (for Anthropic API)
get_current_datetime_schema = ToolParam(
    {
        "name": "get_current_datetime",
        "description": "Returns the current date and time formatted according to the specified format string. This tool provides the current system time formatted as a string. Use this tool when you need to know the current date and time, such as for timestamping records, calculating time differences, or displaying the current time to users. The default format returns the date and time in ISO-like format (YYYY-MM-DD HH:MM:SS).",
        "input_schema": {
            "type": "object",
            "properties": {
                "date_format": {
                    "type": "string",
                    "description": "A string specifying the format of the returned datetime. Uses Python's strftime format codes. For example, '%Y-%m-%d' returns just the date in YYYY-MM-DD format, '%H:%M:%S' returns just the time in HH:MM:SS format, '%B %d, %Y' returns a date like 'May 07, 2025'. The default is '%Y-%m-%d %H:%M:%S' which returns a complete timestamp like '2025-05-07 14:32:15'.",
                    "default": "%Y-%m-%d %H:%M:%S",
                }
            },
            "required": [],
        },
    }
)


In [ ]:
# Tool execution functions - dispatch tool calls to implementations

import json


def run_tool(tool_name, tool_input):
    """
    Execute a single tool by name with the provided input.
    
    Args:
        tool_name: Name of the tool to execute (e.g., 'get_current_datetime')
        tool_input: Dictionary of arguments for the tool
    
    Returns:
        Output from the tool execution
    """
    # Dispatch to the appropriate tool implementation based on name
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)
    # Additional tools can be added here with elif statements


def run_tools(message):
    """
    Process all tool use requests in a message and return tool result blocks.
    
    Args:
        message: Anthropic Message object that may contain tool_use blocks
    
    Returns:
        List of tool_result blocks ready to add to the conversation
    """
    # Extract all tool use blocks from the message content
    tool_requests = [block for block in message.content if block.type == "tool_use"]
    tool_result_blocks = []

    # Process each tool request
    for tool_request in tool_requests:
        try:
            # Execute the tool and get the output
            tool_output = run_tool(tool_request.name, tool_request.input)
            
            # Build a successful tool result block
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": json.dumps(tool_output),
                "is_error": False,
            }
        except Exception as e:
            # Build an error tool result block if execution fails
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": f"Error: {e}",
                "is_error": True,
            }

        tool_result_blocks.append(tool_result_block)

    return tool_result_blocks


In [ ]:
# Main conversation loop that handles tool use and iterative interactions

def run_conversation(messages):
    """
    Run an iterative conversation with the model, handling tool use requests.
    
    This function implements a loop that:
    1. Sends the current message history to the model
    2. Displays the model's text response
    3. If the model requests tools, executes them and continues the loop
    4. Stops when the model finishes (no more tool use requests)
    
    Args:
        messages: List of message dicts (will be modified in place)
    
    Returns:
        Updated message history after conversation ends
    """
    while True:
        # Send the message history to the model with available tools
        response = chat(messages, tools=[get_current_datetime_schema])

        # Add the model's response to the conversation history
        add_assistant_message(messages, response)
        
        # Print the text content of the response
        print(text_from_message(response))

        # Check if the model is requesting tool use or if it has finished
        if response.stop_reason != "tool_use":
            # Model finished (no more tool requests), exit the loop
            break

        # The model requested tool use, so execute the tools
        tool_results = run_tools(response)
        
        # Add the tool results back to the conversation for the next iteration
        add_user_message(messages, tool_results)

    return messages


In [ ]:
# Test the conversation system with a sample user query

# Initialize a new conversation
messages = []

# Add a user query that will require the model to use the get_current_datetime tool
add_user_message(
    messages,
    "What is the current time in HH:MM format? Also, what is the current time in SS format?",
)

# Run the conversation loop, which will handle tool requests automatically
run_conversation(messages)


I can help you get the current time in the requested formats. Let me fetch that for you.

The current time in HH:MM format is 11:16.
The current time in SS format (seconds only) is 30.


[{'role': 'user',
  'content': 'What is the current time in HH:MM format? Also, what is the current time in SS format?'},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text='I can help you get the current time in the requested formats. Let me fetch that for you.', type='text'),
   ToolUseBlock(id='toolu_013M9HM18jyBXPLTTWieH2GV', input={'date_format': '%H:%M'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_013M9HM18jyBXPLTTWieH2GV',
    'content': '"11:16"',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='toolu_01Dv5RUxjEpxCzav4ZAseRVZ', input={'date_format': '%S'}, name='get_current_datetime', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01Dv5RUxjEpxCzav4ZAseRVZ',
    'content': '"30"',
    'is_error': False}]},
 {'role': 'assistant',
  'content': [TextBlock(citations=None, text='The current time 